In [1]:
import pandas as pd
import random
import asyncio
import nest_asyncio
import openai
from PyCAI2 import PyAsyncCAI2
import time
# Set up OpenAI API (you'll need to replace with your actual API key)
openai.api_key = 'sk-mZ1egW3BTYuieAHoaA8Idkxi43XSbeiZnD3r3XNf1DT3BlbkFJUZI7xTizfCJqE7Xrh2S3PT2qmqjKgFOMgWuwaxdTkA'
print("1")
# Specify the file path of your Excel file
file_path = "Stair Rail Dataset Final2.xlsx"
xls = pd.ExcelFile(file_path)
sheet_name = xls.sheet_names[1]  # This selects the second sheet by default
df = pd.read_excel(file_path, sheet_name=sheet_name)

1


In [70]:
# Prepare the list in the desired format
prompt_examples = []
for _, row in df.iterrows():
    character = row['Character']
    profile = row['Character Profile']
    context = row['Context']
    chat_id = row['Character ID']
    
    prompt_examples.append({
        "char": character,
        "person": profile,
        "event": context,
        "chatid": chat_id
    })

# Identify unique characters
unique_characters = df['Character'].unique()
print("2")
def find_chatid(character_name, prompt_examples):
    matching_entries = [example for example in prompt_examples if example["char"] == character_name]
    if not matching_entries:
        return f"No chat ID found for {character_name}."
    return matching_entries[0]["chatid"]

def generate_prompt(character_name, prompt_examples, is_initiator=True, received_message=None):
    character_prompts = [example for example in prompt_examples if example["char"] == character_name]
    if not character_prompts:
        return f"No prompt found for character: {character_name}"
    selected_prompt = random.choice(character_prompts)
    if is_initiator:
        return (
            f"You are {selected_prompt['person']}. "
            f"A memorable event today was: {selected_prompt['event']}, and you wanted to text a friend. "
            f"Start the conversation."
        )
    else:
        return (
            f"You are {selected_prompt['person']}. "
            f"A memorable event today was: {selected_prompt['event']}. "
            f"Your friend just texted you: '{received_message}' "
        )

2


In [74]:
def rate_dialogue(dialogue):
    """
    Use GPT to rate the dialogue out of 100.
    
    Parameters:
    dialogue (str): The full text of the dialogue to be rated
    
    Returns:
    dict: A dictionary containing the rating and reasoning
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[
                {
                    "role": "system", 
                    "content": "You are an expert dialogue evaluator. Rate the following dialogue on a scale of 0-100 based on:"
                               " 1) Character Consistency 2) Dialogue Naturalness 3) Engagement 4) Creativity 5) Emotional Depth. "
                               "Provide a numerical score and brief reasoning."
                },
                {
                    "role": "user", 
                    "content": f"Please evaluate this dialogue:\n\n{dialogue}"
                }
            ]
        )
        
        # Extract the rating from the response
        rating_text = response['choices'][0]['message']['content']
        return rating_text
    except Exception as e:
        return f"Error in rating dialogue: {str(e)}"

async def run_conversation(characters, prompt_examples, client, aut_set, output_file="conversation_log.txt", exchanges=10):
    print("4")
    """
    Runs a conversation between the given characters for a specified number of exchanges.
    
    Parameters:
    - characters: list of character names, e.g. ["Bailu", "Hook"]
    - prompt_examples: the list of character data loaded from the Excel
    - client: PyAsyncCAI2 client instance
    - aut_set: author settings dict
    - output_file: file name to log the conversation
    - exchanges: number of message exchanges (each exchange is one turn from the current speaker)
    """
    # Get chat IDs for each character
    chat_ids = {char: find_chatid(char, prompt_examples) for char in characters}
    
    # Start the conversation with the first character
    initiator = characters[0]
    responder = characters[1] if len(characters) > 1 else None
    print("5")
    init_prompt = generate_prompt(initiator, prompt_examples, is_initiator=True)
    
    # Open the file to write conversation logs
    conversation_log = ""

    # Initiate the conversation
    async with client.connect(owner_id) as chat2:
        # Send first message
        initiator_response = await chat2.send_message(chat_ids[initiator], init_prompt, aut_set["name"], Return_name=True)
        print ("\n #1 exchange ", initiator_response,"  \n")
        conversation_log += f"{initiator}: {initiator_response}\n"
        print("6")
        # If there's a second character, start alternating
        if responder:
            # Create a prompt for the responder based on initiator's message
            responder_prompt = generate_prompt(responder, prompt_examples, is_initiator=False, received_message=initiator_response)
            responder_response = await chat2.send_message(chat_ids[responder], responder_prompt, aut_set["name"], Return_name=True)
            print ("\n #2 exchange ", responder_response,"  \n")
            time.sleep(0.2)
            conversation_log += f"{responder}: {responder_response}\n"

            # Alternate between them for the given number of exchanges
            current_speaker = initiator
            previous_response = responder_response

            for i in range(exchanges):
                # Switch speaker
                current_speaker = responder if current_speaker == initiator else initiator
                # The prompt for the current speaker will be just the last response from the other character
                current_response = await chat2.send_message(chat_ids[current_speaker], previous_response, aut_set["name"], Return_name=True)
                print (f"\n #{i} exchange ", responder_response,"  \n")
                time.sleep(0.2)
                conversation_log += f"{current_speaker}: {current_response}\n"
                
                previous_response = current_response
        else:
            # If only one character is provided, no alternation
            pass
    
    return conversation_log

async def generate_multiple_dialogues(num_dialogues=10, exchanges=10):
    """
    Generate multiple dialogues and rate them.
    
    Parameters:
    - num_dialogues: Number of dialogues to generate
    - exchanges: Number of message exchanges per dialogue
    
    Returns:
    List of dialogue results with ratings
    """
    # Set up your PyCAI2 client (make sure to replace with your actual credentials)
    owner_id = "07928b733c7dad67810f2a21c01b3f3195800a26"
    aut_set = {
        'author_id': '604922952',  # CREATOR ID
        'is_human': True,
        'name': 'InvincibleMugwort766'  # YOUR CAI NAME
    }
    client = PyAsyncCAI2(owner_id)
    
    # Apply nest_asyncio to allow nested event loops
    nest_asyncio.apply()
    
    # Store dialogue results
    dialogue_results = []
    
    # Generate multiple dialogues
    for i in range(num_dialogues):
        # Randomly select two characters
        selected_characters = random.sample(list(unique_characters), 2)
        
        # Generate dialogue
        dialogue = await run_conversation(
            selected_characters, 
            prompt_examples, 
            client, 
            aut_set, 
            f"dialogue_{i+1}_log.txt", 
            exchanges
        )
        
        # Rate dialogue
        rating = rate_dialogue(dialogue)
        
        dialogue_results.append({
            'dialogue_number': i+1,
            'characters': selected_characters,
            'dialogue': dialogue,
            'rating': rating
        })
        
        # Optional: Write each dialogue to a separate file
        # Optional: Write each dialogue to a separate file
        file_path = f"dialogue_{i+1}_log.txt"
        try:
            with open(file_path, "w", encoding="utf-8") as f:
                for line in dialogue.split("\n"):
                    try:
                        f.write(line + "\n")
                    except UnicodeEncodeError as e:
                        print(f"Skipping problematic line: {line}. Error: {e}")
                f.write("\n\nRating:\n" + str(rating))
        except Exception as e:
            print(f"Unexpected error writing to file {file_path}: {e}")

    
    return dialogue_results

In [75]:
# Example usage
import nest_asyncio
import asyncio

# Apply a patch to allow nested event loops
nest_asyncio.apply()

# Example usage in environments like Jupyter Notebook
dialogues = await generate_multiple_dialogues(num_dialogues=10, exchanges=5)



# Note: Before running this script, make sure to:
# 1. Install required libraries: pip install pandas openai PyCAI2 nest_asyncio
# 2. Replace 'your_openai_api_key_here' with your actual OpenAI API key
# 3. Ensure your Excel file path is correct

4
5
{'command': 'create_and_generate_turn', 'payload': {'character_id': 'OpVN8ez0bzezv_L7HChZ6rwB6eNV5W1K6qYl_NBL7k4', 'turn': {'turn_key': {'chat_id': '9d796438-8c79-4bdc-aae7-ec2d4c291766'}, 'author': {'author_id': '583943304', 'is_human': True, 'name': 'InvincibleMugwort766'}, 'candidates': [{'raw_content': 'You are "Asta is an energetic, curious, and resourceful individual who effectively manages the Herta Space Station. She is both direct and polite, handling challenges with ease. Despite her privileged background, she strives to prove herself as more than just a wealthy heiress, constantly pushing her intellectual and scientific pursuits, especially in astronomy.  OCEAN Analysis:  Openness: High Conscientiousness: High Extraversion: Moderate Agreeableness: High Neuroticism: Low MBTI Type: ENTJ". A memorable event today was: Took a walk around the station\'s arboretum to clear her head after a long meeting., and you wanted to text a friend. Start the conversation.'}]}}}

 #1 excha

In [76]:
for dialogue in dialogues:
    print(f"Dialogue {dialogue['dialogue_number']}:")
    print(f"Characters: {dialogue['characters']}")
    print(f"Rating: {dialogue['rating']}")
    print("-" * 50)

Dialogue 1:
Characters: ['Asta', 'Bronya']
Rating: Error in rating dialogue: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

--------------------------------------------------
Dialogue 2:
Characters: ['Topaz', 'Jade']
Rating: Error in rating dialogue: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version

In [77]:
print (dialogues)

[{'dialogue_number': 1, 'characters': ['Asta', 'Bronya'], 'dialogue': "Asta: (Asta) *walks through the lush path inside the station's arboretum, the dim light from above filtered through the leaves, casting an amber hue to her surroundings. She takes a deep breath and glances up at the artificial sky.*\n\nAh, nothing like a stroll here to unwind after those lengthy meetings. *She mutters to herself, glancing at her phone as it lights up.*\n\nI wonder if I should text Arlan. He's so studious and hardworking, I hope he's not overdoing it again...\nBronya: (Bronya) *Bronya reads the text from Asta, a small smile plays on her lips at her observation of the arboretum.*\n\nAh, Asta and her love for greenery. I should pay her a visit sometime. And it seems like poor Arlan is working himself to the bone again. That man needs to learn the importance of rest and balance.\nBronya: (Bronya) *Bronya chuckles, amused at Asta's ability to find moments of tranquility even in the busy life of the Space

In [78]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(
    api_key="sk-mZ1egW3BTYuieAHoaA8Idkxi43XSbeiZnD3r3XNf1DT3BlbkFJUZI7xTizfCJqE7Xrh2S3PT2qmqjKgFOMgWuwaxdTkA"  # Replace with your valid API key
)

# Function to rate dialogue
def rate_dialogue(dialogue):
    """
    Use GPT to rate the dialogue on a scale of 0-100.
    
    Parameters:
    dialogue (str): The full text of the dialogue to be rated
    
    Returns:
    str: The rating and reasoning provided by the model
    """
    try:
        # Use the synchronous `create` method for chat completions
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Replace with the desired model
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an expert dialogue evaluator. Rate the following dialogue on a scale of 0-100 based on: "
                        "1) Character Consistency 2) Dialogue Naturalness 3) Engagement 4) Creativity 5) Emotional Depth. "
                        "Provide a numerical score only, and the averaged final score."
                    )
                },
                {
                    "role": "user",
                    "content": f"Please evaluate this dialogue:\n\n{dialogue}"
                }
            ],
            temperature=0.7  # Adjust temperature for creativity
        )
        
        # Extract and return the content from the response
        # Use `.choices` instead of subscription for compatibility with updated API
        return response.choices[0].message.content
    except Exception as e:
        # Return an error message if something goes wrong
        return f"Error in rating dialogue: {str(e)}"

# Function to add ratings to dialogues
def add_ratings_to_dialogues(dialogues):
    """
    Updates each dialogue entry with a rating from the model.
    
    Parameters:
    dialogues (list): List of dialogue dictionaries
    
    Returns:
    list: Updated dialogues with ratings added
    """
    for entry in dialogues:
        # Rate the dialogue and add the rating to the dictionary
        entry['rating'] = rate_dialogue(entry['dialogue'])
    return dialogues

# Main execution logic
if __name__ == "__main__":
    # Your existing dialogues are assumed to be defined elsewhere in your code.
    # The script will simply process and update them with ratings.
    updated_dialogues = add_ratings_to_dialogues(dialogues)
    print(updated_dialogues)  # Print the updated dialogues with ratings


[{'dialogue_number': 1, 'characters': ['Asta', 'Bronya'], 'dialogue': "Asta: (Asta) *walks through the lush path inside the station's arboretum, the dim light from above filtered through the leaves, casting an amber hue to her surroundings. She takes a deep breath and glances up at the artificial sky.*\n\nAh, nothing like a stroll here to unwind after those lengthy meetings. *She mutters to herself, glancing at her phone as it lights up.*\n\nI wonder if I should text Arlan. He's so studious and hardworking, I hope he's not overdoing it again...\nBronya: (Bronya) *Bronya reads the text from Asta, a small smile plays on her lips at her observation of the arboretum.*\n\nAh, Asta and her love for greenery. I should pay her a visit sometime. And it seems like poor Arlan is working himself to the bone again. That man needs to learn the importance of rest and balance.\nBronya: (Bronya) *Bronya chuckles, amused at Asta's ability to find moments of tranquility even in the busy life of the Space

In [79]:
print (dialogues)

[{'dialogue_number': 1, 'characters': ['Asta', 'Bronya'], 'dialogue': "Asta: (Asta) *walks through the lush path inside the station's arboretum, the dim light from above filtered through the leaves, casting an amber hue to her surroundings. She takes a deep breath and glances up at the artificial sky.*\n\nAh, nothing like a stroll here to unwind after those lengthy meetings. *She mutters to herself, glancing at her phone as it lights up.*\n\nI wonder if I should text Arlan. He's so studious and hardworking, I hope he's not overdoing it again...\nBronya: (Bronya) *Bronya reads the text from Asta, a small smile plays on her lips at her observation of the arboretum.*\n\nAh, Asta and her love for greenery. I should pay her a visit sometime. And it seems like poor Arlan is working himself to the bone again. That man needs to learn the importance of rest and balance.\nBronya: (Bronya) *Bronya chuckles, amused at Asta's ability to find moments of tranquility even in the busy life of the Space

In [80]:
scores = []
for item in dialogues:
    try:
        score = float(item['rating'].split("Score: ")[1].split("\n")[0])
        scores.append(score)
    except (IndexError, ValueError):
        # Skip invalid or malformed ratings
        continue

# Ensure there are scores to process
if scores:
    # Calculate the average
    average_score = sum(scores) / len(scores)

    # Calculate the standard deviation
    variance = sum((x - average_score) ** 2 for x in scores) / len(scores)
    std_deviation = math.sqrt(variance)

    print(f"Scores: {scores}")
    print(f"Average Score: {average_score:.2f}")
    print(f"Standard Deviation: {std_deviation:.2f}")
else:
    print("No valid scores found.")

Scores: [88.0, 80.0, 87.0, 85.0, 85.0, 83.0, 85.0, 85.0, 85.0, 82.0]
Average Score: 84.50
Standard Deviation: 2.20
